In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pickle

def load_graph(file_path):
    with open(file_path, 'rb') as f:
        graph = pickle.load(f)
    return graph

def show_graph(graph):
    print(graph)

    plt.figure(figsize=(5, min(graph.number_of_nodes()/3, 60)))
    pos = nx.nx_pydot.pydot_layout(graph, prog='dot')
    labels = {node: node[:7] for node in graph.nodes}
    nodeColors = [
        data.get("color", "gray")
        for _, data in graph.nodes(data=True)
    ]
    
    nx.draw(graph,pos, labels=labels,  font_size=8, node_color= nodeColors)

    plt.title("Commit Graph")
    plt.show()



In [3]:
import importlib
import statsLib as slib
importlib.reload(slib)
 
st = slib.Stats("dossiers_git")
#st.show_chart_by_stars(st.get_infos_by_stars("forks_count").normalize().div(st.get_by_stars(st.nb_commit)))
#st.show_chart_by_stars(st.get_by_stars(st.degree_entrant))
#st.show_chart_by_stars(st.get_by_stars(st.nb_merges).div(st.get_by_stars(st.nb_commit)))
#st.show_chart_by_stars(st.get_by_stars(st.nb_merges_diff_auteurs))
#st.show_chart_by_stars(st.get_by_stars(st.connexe))
H = nx.DiGraph()
H.add_edges_from([(1, 2), (2, 3), (1,3)])
#st.show_chart_by_stars(st.get_by_stars(st.nb_motifs, H, "A"))



In [22]:
#st.show_multiple_charts_by_stars(st.get_infos_by_stars("forks_count").normalize(),
#                                 st.get_infos_by_stars("open_issues_count").normalize(),
#                                 st.get_infos_by_stars("size").normalize(),
#                                 st.get_by_stars(st.nb_commit).normalize(),
#                                 st.get_by_stars(st.nb_merges).normalize(),
#                                 st.get_by_stars(st.nb_edges).normalize(),
#                                 st.get_by_stars(st.connexe).normalize())

In [4]:
def nb_requetes(nbReposPerStars:int, stars_list:list[int]):
    return (1+(nbReposPerStars-1)//100)*(len(stars_list)-1)

nb_requetes(10, [0, 10, 80, 270, 640, 1250, 2160, 3430, 5120, 7290])

9

In [42]:
#st.show_infos()

In [5]:
[10*i*i*i for i in range(10)]
# 50 "[0, 10, 80, 270, 640, 1250, 2160, 3430, 5120, 7290]"

[0, 10, 80, 270, 640, 1250, 2160, 3430, 5120, 7290]

In [ ]:
def nb_motifs(G, H):
    matcher = nx.algorithms.isomorphism.DiGraphMatcher(G, H)
    return sum(1 for _ in matcher.subgraph_isomorphisms_iter())

H = nx.DiGraph()
H.add_edges_from([(1, 2), (2, 3), (1,3)])

g_test = load_graph("dossiers_git/graphs/10-80/18231296")
nb_motifs(g_test,H)

[{'9f13253': 1, 'caf47d2': 2, '369fa44': 3}, {'ae760eb': 1, 'a273568': 2, '906a849': 3}]


2

In [3]:
g_test = load_graph("dossiers_git/graphs/10-80/18231296")

#show_graph(g_test)


In [1]:
import networkx as nx
import itertools
import math
import numpy as np

def show_graphs_grid(graphs, columns=5):
    rows = math.ceil(len(graphs) / columns)
    fig, axes = plt.subplots(rows, columns, figsize=(3*columns, 3*rows))
    axes = axes.flatten()

    colors = plt.get_cmap("tab10").colors

    for i, graph in enumerate(graphs):
        ax = axes[i]

        if len(graph.edges())>= 5:
            pos = nx.nx_pydot.pydot_layout(graph, prog='sfdp')
        else:
            pos = nx.nx_pydot.pydot_layout(graph, prog='dot')

        color = colors[i % len(colors)]

        nx.draw(
            graph, pos, ax=ax,
            with_labels=False,
            node_size=300,
            font_size=8,
            node_color=[color]*len(graph.nodes),
        )
        ax.set_title(f"motif {i+1}", fontsize=10)
        ax.axis('off')

    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

def matrix_to_graph(matrix):
    G = nx.DiGraph()
    n = len(matrix)
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(n):
            if matrix[j][i] == 0 and  matrix[i][j] == 1:
                G.add_edge(i, j)
            elif matrix[j][i] == 1 and  matrix[i][j] == 1:
                return
    return G

def generate_all_oriented_graphs(n):
    graphs = []
    for bits in itertools.product([0, 1], repeat=n*n):
        matrix = np.array(bits).reshape((n, n))
        G = matrix_to_graph(matrix)
        if G:
            graphs.append(G)
    return graphs

def remove_isomorphic(graphs):
    unique_graphs = []
    for g in graphs:
        if not any(nx.is_isomorphic(g, h) for h in unique_graphs):
            unique_graphs.append(g)
    return unique_graphs

def get_dag(graphs):
    dag  = []
    for g in graphs:
        if nx.is_directed_acyclic_graph(g):
            dag.append(g)
    return dag

oriented_graphs = generate_all_oriented_graphs(3)

nonisomorphic_oriented_graphs = remove_isomorphic(oriented_graphs)

dag = get_dag(nonisomorphic_oriented_graphs)

#show_graphs_grid(dag,3)

